In [ ]:
import numpy as np
import pandas as pd
import os, cv2

# Input data files are available in the read-only "../input/" directory

directory_path = '../Dataset/'
folders_list = []    
    
# Directory listing    
for item in os.listdir(directory_path):
    item_path = os.path.join(directory_path, item)
    if os.path.isfile(item_path):
        folders_list += [item]
    elif os.path.isdir(item_path):
        folders_list += [f'{item}:{len(os.listdir(item_path))}']        

print(f'Folders: {folders_list}')

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Funzione per disegnare i landmarks sulla mano
def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.multi_hand_landmarks  # Lista dei punti chiave della mano
    annotated_image = np.copy(rgb_image)  # Crea una copia dell'immagine originale

    for hand_landmarks in hand_landmarks_list:
        mp.solutions.drawing_utils.draw_landmarks(
            annotated_image,  # Immagine su cui disegnare
            hand_landmarks,  # Punti chiave da disegnare
            mp.solutions.hands.HAND_CONNECTIONS  # Connessioni tra i punti chiave
        )

    return annotated_image

# Inizializza MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Cattura il video dalla fotocamera (cambia '0' per webcam diverse)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Converti l'immagine in formato RGB
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Rileva i punti chiave delle mani
    results = hands.process(rgb_image)

    # Se ci sono mani rilevate, disegna i landmarks
    if results.multi_hand_landmarks:
        annotated_image = draw_landmarks_on_image(frame, results)
    else:
        annotated_image = frame  # Usa il frame originale se non ci sono mani

    # Mostra l'immagine con i landmarks
    cv2.imshow('Hand Landmarks', annotated_image)

    # Uscita premendo 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np
import csv

# Percorso al dataset
dataset_path = '../Dataset/'

# Inizializza MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)

# File CSV per salvare le feature
csv_file = '../KNN/asl_features.csv'

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    # Itera attraverso ogni cartella del dataset (lettere dalla 'a' alla 'z')
    for label in sorted(os.listdir(dataset_path)):
        folder_path = os.path.join(dataset_path, label)
        if os.path.isdir(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                image = cv2.imread(img_path)

                # Converti l'immagine in RGB
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Rileva i landmarks
                results = hands.process(rgb_image)
                if results.multi_hand_landmarks:
                    # Estrai le feature dai landmarks
                    for hand_landmarks in results.multi_hand_landmarks:
                        features = []
                        for landmark in hand_landmarks.landmark:
                            features.extend([landmark.x, landmark.y, landmark.z])
                        # Aggiungi l'etichetta della lettera
                        features.append(label)
                        writer.writerow(features)


##### Suddivisione del dataset in Training e Test Set #####

In [4]:
from sklearn.model_selection import train_test_split

# Carica il dataset
import pandas as pd
data = pd.read_csv('../KNN/asl_features.csv', header=None)
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Etichette (lettere)

# Dividi il dataset (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Ricerca del valore ottimale di k #####

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Prova diversi valori di k
k_values = range(1, 21)
mean_accuracies = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)  # 5-fold cross-validation
    mean_accuracies.append(scores.mean())
    print(f'k={k}, Accuracy: {scores.mean():.4f}')

# Trova il valore di k con la migliore accuratezza
best_k = k_values[np.argmax(mean_accuracies)]
print(f'Miglior valore di k: {best_k}')

##### Valutazione del modello con il valore k trovato #####

In [ ]:
# Inizializza il modello con il miglior valore di k
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

# Valutazione sul test set
accuracy = knn.score(X_test, y_test)
print(f'Accuratezza sul test set: {accuracy:.4f}')

##### Riconoscimento dopo aver cliccato space #####

In [10]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

# Carica il modello KNN addestrato
with open('../Models/knn_model_asl.pkl', 'rb') as f:
    knn = pickle.load(f)

# Inizializza MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Funzione per normalizzare i landmarks
def normalize_features(hand_landmarks):
    x_coords = [landmark.x for landmark in hand_landmarks.landmark]
    y_coords = [landmark.y for landmark in hand_landmarks.landmark]

    # Calcola il bounding box della mano
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)

    # Normalizzazione rispetto al bounding box
    normalized_features = []
    for landmark in hand_landmarks.landmark:
        norm_x = (landmark.x - min_x) / (max_x - min_x)
        norm_y = (landmark.y - min_y) / (max_y - min_y)
        normalized_features.extend([norm_x, norm_y, landmark.z])  # Puoi includere o escludere Z a seconda dei test
    return normalized_features

# Variabile per abilitare/disabilitare la predizione
recognition_active = False

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Converti l'immagine in RGB per Mediapipe
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Disegna i landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if recognition_active:
                # Estrai le feature e fai la predizione
                features = []
                for landmark in hand_landmarks.landmark:
                    features.extend([landmark.x, landmark.y, landmark.z])
                prediction = knn.predict([features])
                # Visualizza la lettera predetta
                cv2.putText(frame, f'Predicted: {prediction[0]}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Mostra il frame con o senza la predizione
    cv2.imshow('Sign Language Recognition', frame)

    # Gestione dei tasti
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Esci dal programma
        break
    elif key == ord(' '):  # Attiva/disattiva il riconoscimento con barra spaziatrice
        recognition_active = not recognition_active
        print(f'Recognition {"activated" if recognition_active else "deactivated"}')

cap.release()
cv2.destroyAllWindows()


c:\Users\danif\anaconda3\envs\ASL\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Recognition activated
